<a href="https://colab.research.google.com/github/aqeeelmirza/hello-world/blob/master/IMDB_Movie_Reviews_Large_Dataset_50k.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install ktrain


     |████████████████████████████████| 25.3MB 128kB/s 
     |████████████████████████████████| 421.8MB 41kB/s 
     |████████████████████████████████| 983kB 40.0MB/s 
     |████████████████████████████████| 245kB 49.9MB/s 
     |████████████████████████████████| 890kB 48.8MB/s 
     |████████████████████████████████| 471kB 50.3MB/s 
     |████████████████████████████████| 51kB 8.3MB/s 
     |████████████████████████████████| 3.9MB 48.7MB/s 
     |████████████████████████████████| 450kB 50.6MB/s 
     |████████████████████████████████| 890kB 44.2MB/s 
     |████████████████████████████████| 3.0MB 45.6MB/s 
     |████████████████████████████████| 1.1MB 11.7MB/s 
  Created wheel for ktrain: filename=ktrain-0.20.2-cp36-none-any.whl size=25268858 sha256=fa7e064a6c977ff6a1a87750cbce399a7c2473d78ff2dbd77718f376b0e292f0
  Stored in directory: /root/.cache/pip/wheels/39/cb/df/46a2a4009de6e87d908d89dabdd78216bf7396eb2150ca1bfc
  Created wheel for keras-bert: filename=keras_bert-0.86.0-cp36-none

In [ ]:
!git clone https://github.com/laxmimerit/IMDB-Movie-Reviews-Large-Dataset-50k.git


Cloning into 'IMDB-Movie-Reviews-Large-Dataset-50k'...
remote: Enumerating objects: 10, done.
remote: Counting objects: 100% (10/10), done.
remote: Compressing objects: 100% (8/8), done.
remote: Total 10 (delta 1), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (10/10), done.


In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
import ktrain
from ktrain import text

In [ ]:
data_train = pd.read_excel('/content/IMDB-Movie-Reviews-Large-Dataset-50k/train.xlsx' , dtype = str)
data_train.head()

,Reviews,Sentiment
0,"When I first tuned in on this morning news, I ...",neg
1,"Mere thoughts of ""Going Overboard"" (aka ""Babes...",neg
2,Why does this movie fall WELL below standards?...,neg
3,Wow and I thought that any Steven Segal movie ...,neg
4,"The story is seen before, but that does'n matt...",neg


In [ ]:
data_test = pd.read_excel('/content/IMDB-Movie-Reviews-Large-Dataset-50k/test.xlsx' , dtype = str)
data_test.head()

,Reviews,Sentiment
0,Who would have thought that a movie about a ma...,pos
1,After realizing what is going on around us ......,pos
2,I grew up watching the original Disney Cindere...,neg
3,David Mamet wrote the screenplay and made his ...,pos
4,"Admittedly, I didn't have high expectations of...",neg


In [ ]:
import pandas as pd
(x_train, y_train), (x_test, y_test), preprocess = text.texts_from_df(train_df = data_train,
                                                                      text_column = 'Reviews',
                                                                      label_columns = 'Sentiment',
                                                                      val_df = data_test,
                                                                      maxlen = 512,
                                                                      preprocess_mode = 'bert')

downloading pretrained BERT model (uncased_L-12_H-768_A-12.zip)...
[██████████████████████████████████████████████████]
extracting pretrained BERT model...
done.

cleanup downloaded zip...
done.

preprocessing train...
language: en


Is Multi-Label? False
preprocessing test...
language: en


In [ ]:
x_train[0].shape

(25000, 512)

In [ ]:
model = text.text_classifier(name='bert',
                             train_data = (x_train, y_train),
                             preproc = preprocess)

Is Multi-Label? False
maxlen is 512
done.


In [ ]:
learner = ktrain.get_learner(model = model,
                             train_data = (x_train, y_train),
                             val_data = (x_test, y_test),
                             batch_size = 6)

In [ ]:
#this might take days to run
# learner.lr_find()
# learner.lr_plot()
# Optimal learning rate of this model is 2e-5

In [ ]:
learner.fit_onecycle(lr = 2e-5, epochs = 10)



begin training using onecycle policy with max lr of 2e-05...
Train on 25000 samples, validate on 25000 samples
Epoch 1/10
25000/25000 [==============================] - 5508s 220ms/sample - loss: 0.2606 - accuracy: 0.8909 - val_loss: 0.1428 - val_accuracy: 0.9489
Epoch 2/10
25000/25000 [==============================] - 5500s 220ms/sample - loss: 0.1557 - accuracy: 0.9422 - val_loss: 0.0878 - val_accuracy: 0.9729
Epoch 3/10
25000/25000 [==============================] - 5494s 220ms/sample - loss: 0.0985 - accuracy: 0.9659 - val_loss: 0.0437 - val_accuracy: 0.9854
Epoch 4/10
25000/25000 [==============================] - 5492s 220ms/sample - loss: 0.0686 - accuracy: 0.9762 - val_loss: 0.0381 - val_accuracy: 0.9888
Epoch 5/10
25000/25000 [==============================] - 5482s 219ms/sample - loss: 0.0593 - accuracy: 0.9797 - val_loss: 0.0240 - val_accuracy: 0.9940
Epoch 6/10
25000/25000 [==============================] - 5482s 219ms/sample - loss: 0.0413 - accuracy: 0.9861 - val_loss:

In [ ]:
learner

NameError: ignored

In [ ]:
predictor = ktrain.get_predictor(learner.model, preprocess)

NameError: ignored

In [ ]:
data = ['this movie was horrible. the plot was really borring',
        'this movie is really sucked and acting is very bad',
        'what a beautiful movie and acting was great']

In [ ]:
predictor.predict(data)

['neg', 'neg', 'pos']

In [ ]:
predictor.save('/content/bert')

In [ ]:
!zip -r /content/bert.zip /content/bert

	zip warning: name not matched: /content/bert

zip error: Nothing to do! (try: zip -r /content/bert.zip . -i /content/bert)
